In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
# مسیرها
csv_path = ".\\bee\\bee_data.csv"          # مسیر فایل CSV شما
images_folder = ".\\bee\\bee_imgs"         # مسیر پوشه تصاویر

# خواندن CSV
df = pd.read_csv(csv_path)

# آماده کردن لیست ها
X = []
y = []

# پردازش هر رکورد
for idx, row in df.iterrows():
    img_name = row['file']    # نام ستون فایل عکس در CSV
    label = row['pollen_carrying']         # نام ستون تارگت در CSV

    img_path = Path(images_folder) / img_name
    img = cv2.imread(str(img_path))
    
    if img is None:
        print(f"تصویر {img_path} پیدا نشد!")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # تبدیل BGR به RGB
    img = cv2.resize(img, (128, 128))          # تغییر اندازه تصاویر به 128x128 (اختیاری)
    img = img / 255.0                          # نرمال‌سازی به [0,1]

    X.append(img)
    y.append(label)

# تبدیل به numpy array
X = np.array(X, dtype=np.float32)
y = np.array(y)

# تقسیم داده به آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("اندازه داده‌ها:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

اندازه داده‌ها:
X_train: (4137, 128, 128, 3)
X_test: (1035, 128, 128, 3)
y_train: (4137,)
y_test: (1035,)


In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

output_activation = 'sigmoid'
loss_fn = 'binary_crossentropy'

# ساخت مدل VGG-like
model = Sequential([
    # Block 1
    Conv2D(8, (3,3), activation='relu', padding='same', input_shape=(128,128,3)),
    BatchNormalization(),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Block 2
    Conv2D(16, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Block 3
    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    # Flatten + Dense layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation=output_activation)
])

# کامپایل مدل
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

# خلاصه مدل
model.summary()

# آموزش مدل
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,   # برای VGG کمی بیشتر بهتره
    batch_size=32
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 128, 128, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 128, 128, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 128, 128, 8)    │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 128, 128, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 64, 64, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,067,465 (4.07 MB)

 Trainable params: 1,067,241 (4.07 MB)

 Non-trainable params: 224 (896.00 B)

Epoch 1/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 0.9923 - loss: 0.1050 - val_accuracy: 0.0039 - val_loss: 4.9123
Epoch 2/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 18s 137ms/step - accuracy: 0.9961 - loss: 0.0272 - val_accuracy: 0.0039 - val_loss: 12.6605
Epoch 3/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - accuracy: 0.9956 - loss: 0.0399 - val_accuracy: 0.9778 - val_loss: 0.1021
Epoch 4/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - accuracy: 0.9942 - loss: 0.0553 - val_accuracy: 0.9961 - val_loss: 0.7643
Epoch 5/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 133ms/step - accuracy: 0.9964 - loss: 0.0440 - val_accuracy: 0.9932 - val_loss: 0.5084
Epoch 6/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 133ms/step - accuracy: 0.9969 - loss: 0.0203 - val_accuracy: 0.9961 - val_loss: 0.0622
Epoch 7/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - accuracy: 0.9966 - loss: 0.0209 - val_accuracy: 0.9961 - val_loss: 0.3214
Epoch 8/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 17s 133ms/step - accuracy: 0.9959 - loss: 

In [8]:
model.save('bee_VGG.keras')